Project 4 

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [145]:
data = pd.read_csv('../data/raw/emdat_extreme_temperature_deaths_2000_2025.csv')
data.head()

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,Country,Subregion,Region,Location,Origin,Associated Types,OFDA/BHA Response,Appeal,Declaration,AID Contribution ('000 US$),Magnitude,Magnitude Scale,Latitude,Longitude,River Basin,Start Year,Start Month,Start Day,End Year,End Month,End Day,Total Deaths,No. Injured,No. Affected,No. Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,2000-0003-BGD,No,nat-met-ext-col,Natural,Meteorological,Extreme temperature,Cold wave,NaN,NaN,BGD,Bangladesh,Southern Asia,Asia,"Barisal, Chittagong, Dhaka, Khulna, Rajshahi, ...",NaN,NaN,No,No,No,NaN,6.40,°C,NaN,NaN,NaN,2000,1,NaN,2000,1.00,NaN,49.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.90,"[{""adm1_code"":575,""adm1_name"":""Barisal""},{""adm...",2003-07-01,2023-09-25
1,2000-0020-IND,No,nat-met-ext-col,Natural,Meteorological,Extreme temperature,Cold wave,NaN,NaN,IND,India,Southern Asia,Asia,"Bihar, Uttar Pradesh, West Bengal provinces",NaN,NaN,No,No,No,NaN,NaN,°C,NaN,NaN,NaN,2000,1,NaN,2000,1.00,NaN,275.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.90,"[{""adm1_code"":1511,""adm1_name"":""West Bengal""},...",2003-07-01,2023-09-25
2,2000-0029-BGR,No,nat-met-ext-col,Natural,Meteorological,Extreme temperature,Cold wave,NaN,NaN,BGR,Bulgaria,Eastern Europe,Europe,"Sofia province, Sofia-city province",NaN,NaN,No,No,No,NaN,NaN,°C,NaN,NaN,NaN,2000,1,NaN,2000,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.90,"[{""adm1_code"":722,""adm1_name"":""Sofia""},{""adm1_...",2003-07-01,2023-09-25
3,2000-0092-EGY,No,nat-met-ext-col,Natural,Meteorological,Extreme temperature,Cold wave,NaN,NaN,EGY,Egypt,Northern Africa,Africa,"Kalyoubia, Shrkia, Fayoum province",NaN,NaN,No,No,No,NaN,2.00,°C,NaN,NaN,NaN,2000,1,NaN,2000,1.00,NaN,3.00,105.00,NaN,NaN,105.00,NaN,NaN,NaN,NaN,NaN,NaN,54.90,"[{""adm1_code"":61518,""adm1_name"":""Fayoum""},{""ad...",2003-07-01,2023-09-25
4,2000-0165-RUS,No,nat-met-ext-col,Natural,Meteorological,Extreme temperature,Cold wave,NaN,NaN,RUS,Russian Federation,Eastern Europe,Europe,Moskva province,NaN,NaN,No,No,No,NaN,NaN,°C,NaN,NaN,NaN,2000,2,NaN,2000,2.00,NaN,13.00,252.00,NaN,NaN,252.00,NaN,NaN,NaN,NaN,NaN,NaN,54.90,"[{""adm1_code"":2535,""adm1_name"":""Moskva""}]",2003-07-01,2023-09-25


In [146]:
cols = ['Start Year','End Year','Disaster Subtype','Country','Total Deaths']
data = data[cols]
data

,Start Year,End Year,Disaster Subtype,Country,Total Deaths
0,2000,2000,Cold wave,Bangladesh,49.00
1,2000,2000,Cold wave,India,275.00
2,2000,2000,Cold wave,Bulgaria,NaN
3,2000,2000,Cold wave,Egypt,3.00
4,2000,2000,Cold wave,Russian Federation,13.00
5,2000,2000,Cold wave,Belarus,NaN
6,2000,2000,Heat wave,Croatia,40.00
7,2000,2000,Cold wave,Ukraine,NaN
8,2000,2000,Heat wave,Pakistan,24.00
9,2000,2000,Heat wave,Greece,27.00


In [147]:
data.shape

(535, 5)

In [148]:
data.isnull().sum().sort_values(ascending=False)

Total Deaths        91
Start Year           0
End Year             0
Disaster Subtype     0
Country              0
dtype: int64

In [149]:
data['Disaster Subtype'].value_counts(normalize=True)

Disaster Subtype
Heat wave                  0.45
Cold wave                  0.40
Severe winter conditions   0.14
Name: proportion, dtype: float64

In [150]:
data.columns

Index(['Start Year', 'End Year', 'Disaster Subtype', 'Country',
       'Total Deaths'],
      dtype='object')

In [151]:
features = ['DisNo.', 'Disaster Subtype',
       'Event Name', 'ISO', 'Country', 'Subregion', 'Region','Location',  'Origin', 'Associated Types', 'OFDA/BHA Response', 'Appeal',
       'Declaration', "AID Contribution ('000 US$)", 'Magnitude',
       'Magnitude Scale', 'Start Year',
       'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day',
       'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless',
       'Total Affected', "Total Damage ('000 US$)", "Total Damage, Adjusted ('000 US$)", 'CPI',
       'Admin Units', 'Entry Date', 'Last Update']

In [152]:
deaths = pd.read_csv('../data/raw/number-of-deaths-per-year.csv')
deaths['deaths'] = deaths['Deaths - Sex: all - Age: all - Variant: estimates'].fillna(
    deaths['Deaths - Sex: all - Age: all - Variant: medium']
)
deaths = deaths.drop([
    'Deaths - Sex: all - Age: all - Variant: estimates',
    'Deaths - Sex: all - Age: all - Variant: medium'
], axis=1)
deaths.head(5)

,Entity,Code,Year,deaths
0,Afghanistan,AFG,1950,290972.00
1,Afghanistan,AFG,1951,288752.00
2,Afghanistan,AFG,1952,288059.00
3,Afghanistan,AFG,1953,287712.00
4,Afghanistan,AFG,1954,289189.00


In [153]:
deaths = deaths[deaths['Year'].between(2020, 2024)]
deaths = deaths.drop(columns= 'Code')
deaths.head()

,Entity,Year,deaths
70,Afghanistan,2020,295555.00
71,Afghanistan,2021,320101.00
72,Afghanistan,2022,243197.00
73,Afghanistan,2023,240296.00
74,Afghanistan,2024,243182.00


In [154]:
population = deaths = pd.read_csv('../data/raw/population.csv',header=1)
population.head(5)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54922.00,55578.00,56320.00,57002.00,57619.00,58190.00,58694.00,58990.00,59069.00,59052.00,58950.00,58781.00,58047.00,58299.00,58349.00,58295.00,58368.00,58580.00,58776.00,59191.00,59909.00,60563.00,61276.00,62228.00,62901.00,61728.00,59931.00,59159.00,59331.00,60443.00,62753.00,65896.00,69005.00,73685.00,77595.00,79805.00,83021.00,86301.00,88451.00,89659.00,90588.00,91439.00,92074.00,93128.00,95138.00,97635.00,99405.00,100150.00,100917.00,101604.00,101838.00,102591.00,104110.00,105675.00,106807.00,107906.00,108727.00,108735.00,108908.00,109203.00,108587.00,107700.00,107310.00,107359.00,107624.00,NaN
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130075728.00,133534923.00,137171659.00,140945536.00,144904094.00,149033472.00,153281203.00,157704381.00,162329396.00,167088245.00,171984985.00,177022314.00,182126556.00,187524135.00,193186642.00,198914573.00,204802976.00,210680842.00,217074286.00,223974122.00,230792729.00,238043099.00,245822010.00,253644643.00,261458202.00,269450407.00,277621771.00,286067346.00,294498625.00,302939121.00,311748681.00,320442961.00,329082707.00,338324002.00,347441809.00,356580375.00,366138524.00,375646235.00,385505757.00,395750933.00,406156661.00,416807868.00,427820358.00,439173286.00,450928044.00,463076637.00,475606210.00,488580707.00,502070763.00,516003448.00,530308387.00,544737983.00,559609961.00,575202699.00,590968990.00,607123269.00,623369401.00,640058741.00,657801085.00,675950189.00,694446100.00,713090928.00,731821393.00,750503764.00,769294618.00,NaN
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,9035043.00,9214083.00,9404406.00,9604487.00,9814318.00,10036008.00,10266395.00,10505959.00,10756922.00,11017409.00,11290128.00,11567667.00,11853696.00,12157999.00,12469127.00,12773954.00,13059851.00,13340756.00,13611441.00,13655567.00,13169311.00,11937581.00,10991378.00,10917982.00,11190221.00,11426852.00,11420074.00,11387818.00,11523298.00,11874088.00,12045660.00,12238879.00,13278974.00,14943172.00,16250794.00,17065836.00,17763266.00,18452091.00,19159996.00,19887785.00,20130327.00,20284307.00,21378117.00,22733049.00,23560654.00,24404567.00,25424094.00,25909852.00,26482622.00,27466101.00,28284089.00,29347708.00,30560034.00,31622704.00,32792523.00,33831764.00,34700612.00,35688935.00,36743039.00,37856121.00,39068979.00,40000412.00,40578842.00,41454761.00,42647492.00,NaN
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,97630925.00,99706674.00,101854756.00,104089175.00,106388440.00,108772632.00,111246953.00,113795019.00,116444636.00,119203521.00,122086536.00,125072948.00,128176494.00,131449942.00,134911581.00,138569918.00,142337272.00,146258576.00,150402616.00,154721711.00,159166518.00,163762473.00,168585118.00,173255157.00,177880746.00,182811038.00,187889141.00,193104347.00,198485027.00,204062274.00,209566031.00,215178709.00,221191375.00,227246778.00,233360104.00,239801875.00,246415446.00,253207584.00,260297834.00,267506298.00,274968446.00,282780717.00,290841795.00,299142845.00,307725100.00,316588476.00,325663158.00,334984176.00,344586109.00,354343844.00,364358270.00,374790143.00,385360349.00,396030207.00,406992047.00,418127845.00,429454743.00,440882906.00,452195915.00,463365429.00,474569351.00,485920997.00,497387180.00,509398589.00,521764076.00,NaN
4,Angola,AGO,"Population, total",SP.POP.TOTL,5231654.00,5301583.00,5354310.00,5408320.00,5464187.00,5521981.00,5581386.00,5641807.00,5702699.00,5763685.00,5852788.00,5991102.00,6174262.00,6388528.00,6613367.00,6842947.00,7074664.00,7317829.00,7576734.00,7847207.00,8133872.00,8435607.00,8751648.00,9

In [155]:
cols = ['Country Name', '2020', '2021', '2022', '2023', '2024']
population = population[cols]
population.head()

,Country Name,2020,2021,2022,2023,2024
0,Aruba,108587.00,107700.00,107310.00,107359.00,107624.00
1,Africa Eastern and Southern,694446100.00,713090928.00,731821393.00,750503764.00,769294618.00
2,Afghanistan,39068979.00,40000412.00,40578842.00,41454761.00,42647492.00
3,Africa Western and Central,474569351.00,485920997.00,497387180.00,509398589.00,521764076.00
4,Angola,33451132.00,34532429.00,35635029.00,36749906.00,37885849.00


In [156]:
population = population.melt(id_vars=['Country Name'], 
                     var_name='Year', 
                     value_name='Population')
population.head()

,Country Name,Year,Population
0,Aruba,2020,108587.00
1,Africa Eastern and Southern,2020,694446100.00
2,Afghanistan,2020,39068979.00
3,Africa Western and Central,2020,474569351.00
4,Angola,2020,33451132.00


In [ ]:
print(deaths.columns)


Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       '2023', '2024', 'Unnamed: 69'],
      dtype='object')


In [142]:
# same names
data['Country'] = data['Country'].str.strip()
data['Year'] = data['Start Year']

population['Country'] = population['Country Name'].str.strip()
population['Year'] = population['Year'].astype(int)

deaths['Country'] = deaths['Entity'].str.strip()
deaths['Year'] = deaths['Year'].astype(int)
weather_pop = weather.merge(population[['Country', 'Year', 'Population']],
                            on=['Country', 'Year'], how='left')

KeyError: 'Entity'

In [ ]:
weather_pop.head()